In [1]:
import re
import requests
import json
import numpy as np
import pandas as pd
import folium
from builtins import round as py_round
import pulp
from math import radians, cos, sin, asin, sqrt
import time
from scipy.spatial import distance_matrix
import networkx as nx
from geopy.distance import geodesic
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from ortools.constraint_solver import pywrapcp
from ortools.linear_solver import pywraplp
from ortools.sat.python import cp_model
from urllib.parse import unquote
from collections import defaultdict
from itertools import combinations
from pulp import PULP_CBC_CMD
from pulp import value

# RETORNAR LISTA DE PRODUROS

In [2]:
def consultarProdutoDesc(gtin,raio, my_lat, my_lon, dias):
    url = 'http://api.sefaz.al.gov.br/sfz-economiza-alagoas-api/api/public/produto/pesquisa'#"http://api.sefaz.al.gov.br/sfz_nfce_api/api/public/consultarPrecosPorDescricao"#

    data={
        "produto": {
            "gtin": gtin
	},
        "estabelecimento": {
            "geolocalizacao": {
                "latitude": my_lat,
                "longitude": my_lon,
                "raio": raio
		}	
	},
        "dias": dias,
        "pagina": 1,
        "registrosPorPagina": 50
}
    
    headers = {
        "Content-Type": "application/json",
        "AppToken":     "ad909a7a6f0d6a130941ae2a9706eec58c0bb65d"
    }
    
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [3]:
#-9.658232850943236, -35.70447119966636 -9.65820182809976, -35.70447142904843 -9.65697, -35.70436
latitude = -9.65697
longitude = -35.70436
dias = 1
raio = 2

In [ ]:
item_list = ['7896006716112', '7893500024996', '7896012300213', '7898018160082', '7896084700027',
             '7896006744115', '7893500007715', '7898383101000', '7898907040969', '7898902735167']#,
             #'7896213005184', '7896532701576', '7896005213018', '7896005030530', '7896005213018',
             #'7896102501872', '7908235000259', '7896036098981', '7897517206086', '7898366932768',
             #'7891107101621', '7892300001428', '7898247780075', '7896036090244', '7892300030060']
            

'''

'7896065200072', '7896215300591', '7896065200065', '7897261800011', '7896215300591',
            '7896005800027', '7896224808101', '7896224803069', '7898286200060', '7896005213018',
            '7896481130106', '7891091010718', '7898366932973', '7898932426042', '7898366930023',
            '7891167021013', '7891167023017', '7891167023024', '7896009301063', '7891167023024',
            '7896024760302', '7896005030349', '7896024721358', '7896061300677', '7891000370896',
            '7896110100012', '7897472200013', '7896110100043', '7898272919150', '7897472200013']
[
    "7893500024996",
    "7898383101000",
    "7896005213018",
    '7896036098981',
    '7892300001428']

7896065200072',
    '7896005213018',
    '7898932426042',
    '7891167023024',
    '7896005030349',
    '7896110100012']
'''

In [ ]:
len(item_list)

In [ ]:

response_list = []

for item in item_list:
    response = consultarProdutoDesc(item, raio, latitude, longitude, dias)
    response_list.append(response)

In [ ]:
data_list = []

for response in response_list:
    for item in response['conteudo']:
        produto = item['produto']
        estabelecimento = item['estabelecimento']
        endereco = estabelecimento['endereco']
        
        data = {
            'CODIGO_BARRAS': int(produto['gtin']),
            'PRODUTO': produto['descricao'],
            'VALOR': produto['venda']['valorVenda'],
            'CNPJ': int(estabelecimento['cnpj']),
            'MERCADO': estabelecimento['razaoSocial'],
            'ENDERECO': endereco['nomeLogradouro'],
            'NUMERO': endereco['numeroImovel'],
            'BAIRRO': endereco['bairro'],
            'LAT': endereco['latitude'],
            'LONG': endereco['longitude']
        }
        
        data_list.append(data)

df = pd.DataFrame(data_list)

In [ ]:
df = df[df['CODIGO_BARRAS'] != 0]

In [ ]:
product_list=df['CODIGO_BARRAS'].unique()

In [ ]:
# Create a unique ID for each market
df['unique_market_id'] = df['CNPJ'].astype(str) + ' - ' + df['MERCADO']

# Get a count of unique barcodes for each market
product_counts = df.groupby('unique_market_id')['CODIGO_BARRAS'].nunique()

# Sort the counts in descending order and get the top 5
ranking_markets = product_counts.sort_values(ascending=False).head(5)

In [ ]:
# Convert the Series to a DataFrame
ranking_markets = ranking_markets.reset_index()

# Get a list of unique_market_ids for the top 5 markets
ranking_markets = ranking_markets['unique_market_id'].tolist()

# Filter df to include only the top 5 markets
df = df[df['unique_market_id'].isin(ranking_markets)]

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
# Initialize the dictionary with the depot information
product_dict = {'00000': {
               'Depot': {
                   'mercado': 'Depot',
                   'produto': 'Depot',
                   'endereco': ' ',
                   'localização': (-9.65697, -35.70436),
                   'valor': 0.0
               }
        # Other relevant details for the depot
    }
}

# Iterate through the rows of the dataframe
for index, row in df.iterrows():
    codigo_barras = str(row['CODIGO_BARRAS'])
    valor = row['VALOR']
    if valor != '0': # Ignore entries with value 0
        mercado_info = {
            row['CNPJ']: {
                "mercado": row['MERCADO'],
                "produto": row['PRODUTO'],
                'endereco': row['ENDERECO'],
                'numero': row['NUMERO'],
                "localização": (row['LAT'], row['LONG']),
                "valor": valor
            }
        }
        
        # If the barcode is already in the dictionary, update the entry. Otherwise, add a new entry.
        if codigo_barras in product_dict:
            product_dict[codigo_barras].update(mercado_info)
        else:
            product_dict[codigo_barras] = mercado_info
            
for key, value in list(product_dict.items())[:2]:
    print(f"{key}: {value}\n")


In [ ]:
G = nx.Graph()

# Adicionando o vértice v0 para representar a localização inicial
G.add_node("v0")  # Você pode adicionar atributos aqui para a localização

# Adicionando mercados como vértices
for barcode, markets in product_dict.items():
    for cnpj, details in markets.items():
        G.add_node(cnpj, mercado=details["mercado"], localização=details["localização"])

# Adicionando arestas entre todos os possíveis pares de nós (conexão completa)
node_pairs = combinations(G.nodes(), 2)
G.add_edges_from(node_pairs)

# Create a new figure and axis
fig, ax = plt.subplots()

# Draw the graph using the specified axis
nx.draw(G, with_labels=True, ax=ax)

# Show the plot
plt.show()


In [ ]:
G = nx.Graph()

# Adicionando o vértice v0 para representar a localização inicial
G.add_node("v0")  # Você pode adicionar atributos aqui para a localização

# Adicionando mercados como vértices
for barcode, markets in product_dict.items():
    for cnpj, details in markets.items():
        G.add_node(cnpj, mercado=details["mercado"], localização=details["localização"])

# Adicionando arestas entre todos os possíveis pares de nós (conexão completa)
node_pairs = combinations(G.nodes(), 2)
G.add_edges_from(node_pairs)

# Create a copy of the graph to avoid any modification to the original graph
H = G.copy()

# Define the node positions (localização) for drawing the graph
pos = {node: data['localização'] for node, data in H.nodes(data=True) if node != 'v0'}
pos['v0'] = (-9.601654, -35.760414)  # Substitua x e y pelas coordenadas da sua localização inicial

# Create a new figure and axis explicitly
fig, ax = plt.subplots()

# Draw the graph using the specified node positions and the axis
nx.draw(H, pos, with_labels=True, ax=ax)

# Show the plot
plt.show()


In [ ]:
def create_matrices(product_dict):
    # Get the depot location
    depot_location = product_dict['00000']['Depot']['localização']

    # Get all the market locations, excluding the depot
    market_locations = [info['localização'] for markets in product_dict.values() for info in markets.values() if 'localização' in info and info['localização'] != depot_location]

    # Combine depot and market locations
    locations = [depot_location] + market_locations
    n = len(locations)
    vehicle_consumption = 9.5 # km per liter
    fuel_price_per_lt = 5.44
    
    # Create the distance and fuel cost matrix
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                distance = geodesic(locations[i], locations[j]).kilometers
                fuel_cost = distance * (1 / vehicle_consumption) * fuel_price_per_lt
                distance_matrix[i][j] = distance + fuel_cost

    # Create indices for the markets
    market_index = {}
    idx = 1 # Start at 1 to exclude the depot
    for markets in product_dict.values():
        for market_cnpj in markets.keys():
            if market_cnpj != 'Depot' and market_cnpj not in market_index:
                market_index[market_cnpj] = idx
                idx += 1

    # Create the price matrix
    m = len(product_dict) - 1 # Number of products, excluding the depot
    corrected_price_matrix = np.full((n, m), float('inf'))
    for product_idx, (product_code, markets_info) in enumerate([item for item in product_dict.items() if item[0] != '00000']):
        for market_cnpj, product_info in markets_info.items():
            corrected_price_matrix[market_index[market_cnpj]][product_idx] = product_info['valor']

    return corrected_price_matrix, distance_matrix




In [ ]:
price_matrix, distance_matrix = create_matrices(product_dict)

In [ ]:
price_matrix

In [ ]:
distance_matrix

In [ ]:
def create_market_index_mapping(product_dict):
    market_index_mapping = {}
    index = 1  # Start with 1 to exclude the depot
    for markets in product_dict.values():
        for market_cnpj, details in markets.items():
            if market_cnpj != 'Depot':
                market_index_mapping[market_cnpj] = details
                market_index_mapping[market_cnpj]['index'] = index
                index += 1
    return market_index_mapping


In [ ]:
print(create_market_index_mapping(product_dict))

In [ ]:
def calculate_markets_to_buy(solution, z, m, n, market_index_mapping):
    markets_to_buy = {}
    for k in range(m):
        for i in range(1, n):
            if solution.get_value(z[k][i]) == 1:
                market_cnpjs = [cnpj for cnpj, details in market_index_mapping.items() if details['index'] == i]
                if market_cnpjs:
                    market_cnpj = market_cnpjs[0]
                    markets_to_buy[k] = [market_cnpj]
                else:
                    raise ValueError(f"Index {i} not found in market_index_mapping")
    return markets_to_buy

def process_route_and_markets(markets_to_buy, market_index_mapping):
    visited_markets = set()
    route_names = ["Depot"]
    for k, markets in markets_to_buy.items():
        market_cnpj = markets[0]
        if market_cnpj not in visited_markets:
            market_name = market_index_mapping[market_cnpj]['mercado']
            route_names.append(f"{market_cnpj} - {market_name}")
            visited_markets.add(market_cnpj)
    route_names.append("Depot")
    print("Rota de custo mínimo:", " --> ".join(route_names))



In [ ]:
def calculate_products_and_subtotals(markets_to_buy, market_index_mapping):
    products_by_market = {}
    market_subtotals = {}
    for product_idx, market_cnpj_list in markets_to_buy.items():
        for market_cnpj in market_cnpj_list:
            market_details = market_index_mapping[market_cnpj]
            market_name = market_details['mercado']
            product = market_details['produto']
            price = market_details['valor']
            products_by_market.setdefault(market_cnpj, []).append((product, price))
            market_subtotals[market_cnpj] = market_subtotals.get(market_cnpj, 0) + price
            print(f"Mercado: {market_name}\nProduto: {product}\nValor: {price}")

    for market_cnpj, subtotal in market_subtotals.items():
        print(f"Subtotal para {market_cnpj}: {np.round(subtotal, 2)}")

    return market_subtotals


In [ ]:
def process_solution(solution, market_index_mapping, n, m, z):
    markets_to_buy = {k: [] for k in range(m)}
    for k in range(m):
        for i in range(1, n):  # Exclude the depot
            if solution.get_value(z[k][i]) == 1:
                market_cnpj = [key for key, details in market_index_mapping.items() if details['index'] == i][0]
                markets_to_buy[k].append(market_cnpj)

    market_subtotals = calculate_products_and_subtotals(markets_to_buy, market_index_mapping)

    total_cost_products = sum(subtotal for subtotal in market_subtotals.values())
    return total_cost_products, markets_to_buy


In [ ]:
def main(distance_matrix, price_matrix, product_dict):
    n = len(distance_matrix) # Number of vertices, including depot
    m = len(product_dict) - 1 # Number of products, excluding depot
    c = distance_matrix      # Travel cost
    b = np.transpose(price_matrix) # Buying cost

    # Create the model
    model = Model(name="route-and-buying-optimization")

    # Decision Variables
    x = [[model.binary_var(name=f'x[{i},{j}]') for j in range(n)] for i in range(n)]
    y = [model.binary_var(name=f'y[{i}]') for i in range(n)]
    z = [[model.binary_var(name=f'z[{k},{i}]') for i in range(n)] for k in range(m)]
    u = [model.continuous_var(name=f'u[{i}]') for i in range(1, n)] # excluding depot

    # Objective Function
    objective_terms = []
    for i in range(n):
        for j in range(n):
            if i != j:
                objective_terms.append(c[i][j] * x[i][j])
    for i in range(n):
        for k in range(m):
            if not np.isinf(b[k][i]):
                objective_terms.append(b[k][i] * z[k][i])

    model.minimize(sum(objective_terms))

    # Constraints
    for i in range(n):
        model.add_constraint(sum(x[i][j] for j in range(n) if i != j) == y[i])
    
    for k in range(m):
        model.add_constraint(sum(z[k][i] for i in range(n)) == 1)

    for k in range(m):
        for i in range(n):
            model.add_constraint(z[k][i] <= y[i])

    model.add_constraint(y[0] == 1)

    for i in range(1, n):
        for j in range(1, n):
            if i != j:
                model.add_constraint(u[i-1] - u[j-1] + n * x[i][j] <= n-1)

    for i in range(1, n):
        model.add_constraint(u[i-1] >= 0)

    # Solve the model
    solution = model.solve()
    
    if solution:
        market_index_mapping = create_market_index_mapping(product_dict)
        total_cost_products, markets_to_buy = process_solution(solution, market_index_mapping, n, m, z)
    
        # Calculate the total cost of distance
        total_cost_distance = 0
        for i in range(n):
            for j in range(n):
                if i != j:
                    total_cost_distance += c[i][j] * solution.get_value(x[i][j])
    
        # Update the total cost calculation
        total_cost = total_cost_products + total_cost_distance
    
        print("Total comprado nos mercados:", np.round(total_cost_products, 2))
        print("Custo mínimo (incluindo distâncias e produtos):", np.round(total_cost, 2))



In [ ]:
if __name__ == '__main__':
    main(distance_matrix, price_matrix, product_dict)

In [ ]:
distance_matrix_str = """[Your copied distance matrix]"""
distance_matrix = np.array([list(map(float, row.split(','))) for row in distance_matrix_str.strip().split('\n')])

# Defining the price_matrix (Example)
price_matrix = [
    [10, 20, 30], # Product 0
    [20, 30, 40], # Product 1
    # Add more if needed
]

# Defining the product_dict (Example)
product_dict = {
    'product_0': [0, 1, 2],
    'product_1': [0, 2],
    # Add more if needed
}

# Running the main function
main(distance_matrix, price_matrix, product_dict)

In [ ]:
distance_matrix = np.array(distance_matrix)

# Printing the shape to ensure it's correctly formed
distance_matrix.shape